In [1]:
from Train import train
from DynamicsModel import model as DynamicsModel
from env1.env1 import InvertedPendulum as env1
import jax 
import jax.numpy as jnp
from sklearn.model_selection import train_test_split
import orbax.checkpoint as ocp
import os 


In [4]:
# Load Data:
with jnp.load('Data/data.npz') as data:
    X_data1 = data['observation_t']
    X_data2 = data['action_t']
    X_data = jnp.concatenate((X_data1, X_data2), axis=1)
    y_data = data['observation_tp']
    print(X_data.shape, y_data.shape)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0)

rng = jax.random.PRNGKey(0)

(1782000, 5) (1782000, 4)
[[-4.402749   -3.3945      0.77425027 -3.7951906  -2.85725   ]
 [-4.3489575  -3.4808223   1.5320121  -1.6269966   2.1163223 ]
 [-4.2681317  -3.4908636   2.1815755   0.19445716  1.8793108 ]
 [-4.179244   -3.4706311   2.2353868   0.605529    0.08415771]
 [-4.108277   -3.4793782   1.6211463  -0.4862598  -1.9696319 ]
 [-4.024826   -3.4480648   2.2401025   1.1937723   1.8161194 ]
 [-3.933969   -3.3917592   2.2813146   1.4723125   0.07477856]
 [-3.8513706  -3.3486323   1.9925027   0.94445825 -0.9096937 ]
 [-3.7710836  -3.3058693   2.011596    1.1055242   0.0316143 ]
 [-3.7002308  -3.281488    1.6912901   0.44514883 -0.97768736]]


In [3]:
params = train(rng, X_train, y_train, X_test, y_test, observation_size = 4,action_size = 1, learning_rate=0.001, num_epochs=200, batch_size=64)

Epoch 0, Train Loss: 0.3950, Test Loss: 0.3945
Epoch 10, Train Loss: 0.3662, Test Loss: 0.3655
Epoch 20, Train Loss: 0.3658, Test Loss: 0.3651
Epoch 30, Train Loss: 0.3654, Test Loss: 0.3647


KeyboardInterrupt: 

In [ ]:
env1 = env1()
key = jax.random.key(0)

State = env1.reset(key)
State1 = env1.step(State, 1.0)
print(State.pipeline_state)
State.pipeline_state.q = jnp.array([0.0,0.0],dtype=jnp.float32)
# qqd = jnp.concatenate((jnp.expand_dims(State.pipeline_state.q,axis=1),jnp.expand_dims(State.pipeline_state.qd, axis=1)), axis=0)[:,0]
# qqd10 = jnp.concatenate((jnp.expand_dims(State1.pipeline_state.q,axis=1),jnp.expand_dims(State1.pipeline_state.qd, axis=1)), axis=0)

# model = DynamicsModel(4,5)
# input = jnp.concatenate((qqd, jnp.array([1.0])), axis=0)
# qqd11 = model.apply({"params":params}, input )



11
State(q=Array([-4.4427404, -3.2601416], dtype=float32), qd=Array([ 1.6889596, -1.9550669], dtype=float32), x=Transform(pos=Array([[-4.4427404,  0.       ,  0.       ],
       [-4.4427404,  0.       ,  0.       ]], dtype=float32), rot=Array([[-1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.05923977,  0.        ,  0.9982438 , -0.        ]],      dtype=float32)), xd=Motion(ang=Array([[ 0.      ,  0.      ,  0.      ],
       [ 0.      , -1.955067,  0.      ]], dtype=float32), vel=Array([[1.6889596, 0.       , 0.       ],
       [1.6889596, 0.       , 0.       ]], dtype=float32)), contact=None, root_com=Array([[-4.431406  ,  0.        , -0.09653018],
       [-4.431406  ,  0.        , -0.09653018]], dtype=float32), cinr=Inertia(transform=Transform(pos=Array([[-0.11869191,  0.        ,  1.0108616 ],
       [ 0.11869382,  0.        , -1.0108615 ]], dtype=float32), rot=Array([[-0.70710677,  0.        , -0.70710677,  0.        ],
       [ 0.9982928 ,  0.        , -0.05840788,

FrozenInstanceError: cannot assign to field 'q'

In [ ]:
ckpt_dir = os.path.abspath('./params')
checkpointer = ocp.AsyncCheckpointer(ocp.StandardCheckpointHandler())
path = ocp.test_utils.erase_and_create_empty(ckpt_dir)
checkpointer.save(path / str(1), args=ocp.args.StandardSave(params))
